### KNN Regression

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV  # Hyperparameyer tuning

from sklearn.preprocessing import MinMaxScaler, StandardScaler  #Feature Scaling

from sklearn.neighbors import KNeighborsRegressor

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [22]:
df = pd.read_csv('Train (2).csv')
df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [24]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [25]:
len(df['Item_Identifier'].unique())

1559

In [26]:
df['Item_Identifier'].value_counts()

FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64

In [27]:
import warnings
warnings.filterwarnings("ignore")

#### 1. item weight

In [28]:
df['Item_Weight'].median()

12.6

In [29]:
df['Item_Weight'].fillna(df['Item_Weight'].median(), inplace=True)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


#### 2. Item_Fat_Content

In [31]:
df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [32]:
df['Item_Fat_Content'].replace({'Low Fat':0, 'Regular':1, 'LF':0, 'reg':1, 'low fat':0}, inplace=True)

In [33]:
df['Item_Fat_Content'].value_counts()

0    5517
1    3006
Name: Item_Fat_Content, dtype: int64

#### Item_Type

In [34]:
df['Item_Type'].value_counts()

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

In [35]:
Item_Type_df = pd.get_dummies(df['Item_Type'], prefix='Item_Type')
Item_Type_df

,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8519,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8520,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
8521,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


#### Outlet_Identifier

In [36]:
df['Outlet_Identifier'].value_counts()

OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    528
Name: Outlet_Identifier, dtype: int64

In [37]:
Outlet_Identifier_df = pd.get_dummies(df['Outlet_Identifier'], prefix='Outlet_Identifier')

In [38]:
Outlet_Identifier_df

,Outlet_Identifier_OUT010,Outlet_Identifier_OUT013,Outlet_Identifier_OUT017,Outlet_Identifier_OUT018,Outlet_Identifier_OUT019,Outlet_Identifier_OUT027,Outlet_Identifier_OUT035,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049
0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
8518,0,1,0,0,0,0,0,0,0,0
8519,0,0,0,0,0,0,0,1,0,0
8520,0,0,0,0,0,0,1,0,0,0
8521,0,0,0,1,0,0,0,0,0,0


#### Outlet_Size

In [39]:
df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [40]:
df['Outlet_Size'].replace({'Small':0,"Medium":1, "High":2}, inplace=True)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   int64  
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   float64
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(5), int64(2), object(5)
memory usage: 799.2+ KB


In [42]:
df['Outlet_Size'].value_counts()

1.0    2793
0.0    2388
2.0     932
Name: Outlet_Size, dtype: int64

In [43]:
df['Outlet_Size'].fillna(1, inplace=True)

#### Outlet_Location_Type

In [44]:
df['Outlet_Location_Type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

In [45]:
df['Outlet_Location_Type'].replace({'Tier 1':0, 'Tier 2':1, 'Tier 3':2}, inplace=True)

#### Outlet_Type

In [46]:
df['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [47]:
Outlet_Type_df = pd.get_dummies(df['Outlet_Type'], prefix='Outlet_Type', 
                                drop_first=True)

In [48]:
Outlet_Type_df

,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,1,0,0
1,0,1,0
2,1,0,0
3,0,0,0
4,1,0,0
...,...,...,...
8518,1,0,0
8519,1,0,0
8520,1,0,0
8521,0,1,0


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   int64  
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                8523 non-null   float64
 9   Outlet_Location_Type       8523 non-null   int64  
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(5), int64(3), object(4)
memory usage: 799.2+ KB


In [50]:
df.drop(['Item_Identifier','Item_Type','Outlet_Identifier', 'Outlet_Type'], axis=1, inplace=True)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Weight                8523 non-null   float64
 1   Item_Fat_Content           8523 non-null   int64  
 2   Item_Visibility            8523 non-null   float64
 3   Item_MRP                   8523 non-null   float64
 4   Outlet_Establishment_Year  8523 non-null   int64  
 5   Outlet_Size                8523 non-null   float64
 6   Outlet_Location_Type       8523 non-null   int64  
 7   Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(5), int64(3)
memory usage: 532.8 KB


In [52]:
df_list = [df,Item_Type_df ,Outlet_Identifier_df, Outlet_Type_df]
df = pd.concat(df_list, axis=1)
df

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Item_Outlet_Sales,Item_Type_Baking Goods,Item_Type_Breads,...,Outlet_Identifier_OUT018,Outlet_Identifier_OUT019,Outlet_Identifier_OUT027,Outlet_Identifier_OUT035,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,9.300,0,0.016047,249.8092,1999,1.0,0,3735.1380,0,0,...,0,0,0,0,0,0,1,1,0,0
1,5.920,1,0.019278,48.2692,2009,1.0,2,443.4228,0,0,...,1,0,0,0,0,0,0,0,1,0
2,17.500,0,0.016760,141.6180,1999,1.0,0,2097.2700,0,0,...,0,0,0,0,0,0,1,1,0,0
3,19.200,1,0.000000,182.0950,1998,1.0,2,732.3800,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8.930,0,0.000000,53.8614,1987,2.0,2,994.7052,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,6.865,0,0.056783,214.5218,1987,2.0,2,2778.3834,0,0,...,0,0,0,0,0,0,0,1,0,0
8519,8.380,1,0.046982,108.1570,2002,1.0,1,549.2850,1,0,...,0,0,0,0,1,0,0,1,0,0
8520,10.600,0,0.035186,85.1224,2004,0.0,1,1193.1136,0,0,...,0,0,0,1,0,0,0,1,0,0
8521,7.210,1,0.145221,103.1332,2009,1.0,2,1845.5976,0,0,...,1,0,0,0,0,0,0,0,1,0


In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 37 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Item_Weight                      8523 non-null   float64
 1   Item_Fat_Content                 8523 non-null   int64  
 2   Item_Visibility                  8523 non-null   float64
 3   Item_MRP                         8523 non-null   float64
 4   Outlet_Establishment_Year        8523 non-null   int64  
 5   Outlet_Size                      8523 non-null   float64
 6   Outlet_Location_Type             8523 non-null   int64  
 7   Item_Outlet_Sales                8523 non-null   float64
 8   Item_Type_Baking Goods           8523 non-null   uint8  
 9   Item_Type_Breads                 8523 non-null   uint8  
 10  Item_Type_Breakfast              8523 non-null   uint8  
 11  Item_Type_Canned                 8523 non-null   uint8  
 12  Item_Type_Dairy     

In [54]:
X = df.drop('Item_Outlet_Sales', axis=1)
y = df['Item_Outlet_Sales']

In [55]:
df

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Item_Outlet_Sales,Item_Type_Baking Goods,Item_Type_Breads,...,Outlet_Identifier_OUT018,Outlet_Identifier_OUT019,Outlet_Identifier_OUT027,Outlet_Identifier_OUT035,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,9.300,0,0.016047,249.8092,1999,1.0,0,3735.1380,0,0,...,0,0,0,0,0,0,1,1,0,0
1,5.920,1,0.019278,48.2692,2009,1.0,2,443.4228,0,0,...,1,0,0,0,0,0,0,0,1,0
2,17.500,0,0.016760,141.6180,1999,1.0,0,2097.2700,0,0,...,0,0,0,0,0,0,1,1,0,0
3,19.200,1,0.000000,182.0950,1998,1.0,2,732.3800,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8.930,0,0.000000,53.8614,1987,2.0,2,994.7052,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,6.865,0,0.056783,214.5218,1987,2.0,2,2778.3834,0,0,...,0,0,0,0,0,0,0,1,0,0
8519,8.380,1,0.046982,108.1570,2002,1.0,1,549.2850,1,0,...,0,0,0,0,1,0,0,1,0,0
8520,10.600,0,0.035186,85.1224,2004,0.0,1,1193.1136,0,0,...,0,0,0,1,0,0,0,1,0,0
8521,7.210,1,0.145221,103.1332,2009,1.0,2,1845.5976,0,0,...,1,0,0,0,0,0,0,0,1,0


In [56]:
X

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,...,Outlet_Identifier_OUT018,Outlet_Identifier_OUT019,Outlet_Identifier_OUT027,Outlet_Identifier_OUT035,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,9.300,0,0.016047,249.8092,1999,1.0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
1,5.920,1,0.019278,48.2692,2009,1.0,2,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,17.500,0,0.016760,141.6180,1999,1.0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
3,19.200,1,0.000000,182.0950,1998,1.0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8.930,0,0.000000,53.8614,1987,2.0,2,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,6.865,0,0.056783,214.5218,1987,2.0,2,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8519,8.380,1,0.046982,108.1570,2002,1.0,1,1,0,0,...,0,0,0,0,1,0,0,1,0,0
8520,10.600,0,0.035186,85.1224,2004,0.0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0
8521,7.210,1,0.145221,103.1332,2009,1.0,2,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [116]:
norm_scaler = MinMaxScaler()
X_nor = norm_scaler.fit_transform(X)
# X_nor
X_nor_df = pd.DataFrame(X_nor, columns=X.columns)
X_nor_df

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,...,Outlet_Identifier_OUT018,Outlet_Identifier_OUT019,Outlet_Identifier_OUT027,Outlet_Identifier_OUT035,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,0.282525,0.0,0.048866,0.927507,0.583333,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,0.081274,1.0,0.058705,0.072068,1.000000,0.5,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.770765,0.0,0.051037,0.468288,0.583333,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,0.871986,1.0,0.000000,0.640093,0.541667,0.5,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.260494,0.0,0.000000,0.095805,0.083333,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,0.137541,0.0,0.172914,0.777729,0.083333,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8519,0.227746,1.0,0.143069,0.326263,0.708333,0.5,0.5,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
8520,0.359929,0.0,0.107148,0.228492,0.791667,0.0,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
8521,0.158083,1.0,0.442219,0.304939,1.000000,0.5,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Train Test Split

In [136]:
X_train, X_test, y_train, y_test = train_test_split(X_nor_df, y, 
                                            test_size=0.2, random_state=42)

In [137]:
X_train

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,...,Outlet_Identifier_OUT018,Outlet_Identifier_OUT019,Outlet_Identifier_OUT027,Outlet_Identifier_OUT035,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
549,0.294433,1.0,0.107207,0.594464,0.583333,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
7757,0.800536,0.0,0.144563,0.591057,0.708333,0.5,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
764,0.776719,1.0,0.231802,0.341387,0.500000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
6867,0.224472,0.0,0.090883,0.043819,0.708333,0.5,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2716,0.493897,0.0,0.417880,0.527478,0.500000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,0.288181,1.0,0.871963,0.457956,0.541667,0.5,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5191,0.657636,0.0,0.358032,0.188359,0.916667,0.5,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5390,0.776719,0.0,0.057686,0.874662,0.708333,0.5,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
860,0.940458,0.0,0.165543,0.367815,0.916667,0.5,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [138]:
X_test

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,...,Outlet_Identifier_OUT018,Outlet_Identifier_OUT019,Outlet_Identifier_OUT027,Outlet_Identifier_OUT035,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
7503,0.580232,0.0,0.080087,0.204332,0.083333,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2957,0.200953,0.0,0.216619,0.048466,0.500000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
7031,0.592141,1.0,0.125805,0.045651,0.583333,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1084,0.479012,1.0,0.136322,0.604484,0.000000,0.5,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
856,0.335814,1.0,0.037930,0.705527,0.791667,0.0,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7205,0.431378,1.0,0.285196,0.406965,0.500000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3257,0.146770,0.0,0.000000,0.496113,1.000000,0.5,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6346,0.592141,1.0,0.125505,0.045651,0.083333,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6318,0.312295,1.0,0.429926,0.081540,1.000000,0.5,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Train Model

In [139]:
knn_reg = KNeighborsRegressor(n_neighbors=10, p=2)
knn_reg.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=10)

In [140]:
y_pred = knn_reg.predict(X_test)
y_pred

array([1395.05074, 1357.76594, 1451.31084, ..., 1265.28632, 1131.12762,
       1442.05622])

In [141]:
len(y_pred)

1705

In [142]:
y_test[:5]

7503    1743.0644
2957     356.8688
7031     377.5086
1084    5778.4782
856     2356.9320
Name: Item_Outlet_Sales, dtype: float64

In [143]:
y_pred[:5]

array([1395.05074, 1357.76594, 1451.31084, 4937.5728 , 3190.77992])

In [144]:
accuracy = r2_score(y_test, y_pred)
print("Accuarcy of knn model is :", accuracy)

Accuarcy of knn model is : 0.4997279746249026


### Evaluation

In [145]:
# Mean Squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error is :", mse)

Mean Squared Error is : 1359724.3901352359


In [146]:
# Root Mean Squared error
rmse = np.sqrt(mse)
print("Root Mean Squared Error is :", rmse)

Root Mean Squared Error is : 1166.0722062270568


In [147]:
# Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error is :", mae)

Mean Absolute Error is : 846.2186011495601


In [148]:
# Accuracy score
accuracy = r2_score(y_test, y_pred)
print("Accuarcy of knn model is :", accuracy)

Accuarcy of knn model is : 0.4997279746249026


### Hyperparameter Tuning

#### Grid SearchCV

In [150]:
k = np.arange(7,20)
p = [1,2]
hyp = {'n_neighbors': k, "p":p}

In [151]:
knn = KNeighborsRegressor()
best_knn_model = GridSearchCV(knn,hyp,cv=5)
best_knn_model.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
                         'p': [1, 2]})

In [152]:
y_pred1 = knn_reg.predict(X_test)
y_pred1

array([1395.05074, 1357.76594, 1451.31084, ..., 1265.28632, 1131.12762,
       1442.05622])

In [153]:
best_knn_model.best_params_

{'n_neighbors': 7, 'p': 2}

In [154]:
# Accuracy score
accuracy = r2_score(y_test, y_pred1)
print("Accuarcy of knn model is :", accuracy)

Accuarcy of knn model is : 0.4997279746249026


#### Randomized Search CV

In [102]:
knn = KNeighborsRegressor()
best_knn_model = RandomizedSearchCV(knn,hyp,cv=5)
best_knn_model.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsRegressor(),
                   param_distributions={'n_neighbors': array([ 7,  8,  9, 10, 11, 12, 13]),
                                        'p': [1, 2]})

In [103]:
best_knn_model.best_params_

{'p': 2, 'n_neighbors': 7}

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                            test_size=0.2, random_state=42)

In [105]:
knn_reg1 = KNeighborsRegressor(n_neighbors=7, p=2)
knn_reg1.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=7)

In [106]:
y_pred2 = knn_reg1.predict(X_test)
y_pred2

array([1249.0408    ,  869.62991429,  742.84257143, ...,  668.74854286,
        584.95285714, 1238.67334286])

In [107]:
# Accuracy score
accuracy = r2_score(y_test, y_pred2)
print("Accuarcy of knn model is :", accuracy)

Accuarcy of knn model is : 0.4985863166820106
